In [2]:
import sys
import json
import base64
import numpy as np
import os
from PIL import Image
import io
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
# from keras.models import load_model
print(tf.__version__)

2.15.0


In [28]:
!pip install -U -q segmentation-models
# !pip install -q numpy==1.18.5
# !pip install -q tensorflow==2.15.0
!pip install -q tensorflow==2.15.0

!pip install -q keras==2.15.0
# !pip install -q keras
!pip install -q tensorflow-estimator



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:

def crop_center(img, cropx, cropy):
    # Assuming img has shape (slices, height, width, channels)
    d, y, x, c = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    
    # Ensure the cropped image has the same number of channels
    return img[:, starty:starty+cropy, startx:startx+cropx, :]

def pad_slices(scan, target_slices= 50):
    # Get the current number of slices
    current_slices = scan.shape[0]
    # Check if padding is necessary
    if current_slices < target_slices:
        # Calculate padding amounts
        pad_before = (target_slices - current_slices) // 2
        pad_after = target_slices - current_slices - pad_before
        # Pad the scan with zeros on the slices axis (axis 0)
        padded_scan = np.pad(scan, pad_width=((pad_before, pad_after), (0, 0), (0, 0)), mode='constant', constant_values=0)
    elif current_slices > target_slices:
        # Calculate the cropping needed
        start = (current_slices - target_slices) // 2
        end = start + target_slices
        # Crop the scan to the target size
        padded_scan = scan[start:end, :, :]
    else:
        # If the number of slices is already equal to the target, no action is needed
        padded_scan = scan
    return padded_scan


In [14]:
im = []
image_array = np.load('uploads/data_a.npy')
image_array_s = np.load('uploads/data_s.npy')
image_array_c = np.load('uploads/data_c.npy')

for i in range(len(image_array)):
    im.append(Image.fromarray(image_array[i].astype('uint8')))
for i in range(len(image_array_s)):
    im.append(Image.fromarray(image_array_s[i].astype('uint8')))
for i in range(len(image_array_c)):
    im.append(Image.fromarray(image_array_c[i].astype('uint8')))


In [16]:
im

[<PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L siz

In [17]:
array_a = pad_slices(image_array)
array_s = pad_slices(image_array_s)
array_c = pad_slices(image_array_c)




In [18]:
all = np.stack((array_a, array_s, array_c), axis=-1)
all = np.array([crop_center(all, 160, 160) ])


In [30]:
for i in range(50):
    print(array_a[25][i])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [31]:

batch_min = all.min()
batch_max = all.max()
all = (all.astype(np.float32) - batch_min) / (batch_max - batch_min)


In [33]:
all.shape
for i in range(50):
    print(all[0][25][i])

[[0.05098039 0.3019608  0.13333334]
 [0.0627451  0.29411766 0.14901961]
 [0.0627451  0.28627452 0.27450982]
 [0.07843138 0.27058825 0.5764706 ]
 [0.0627451  0.27058825 0.7490196 ]
 [0.07058824 0.29411766 0.75686276]
 [0.08627451 0.34117648 0.7607843 ]
 [0.0627451  0.29803923 0.7607843 ]
 [0.05882353 0.26666668 0.6627451 ]
 [0.0627451  0.2784314  0.6392157 ]
 [0.05882353 0.25882354 0.6745098 ]
 [0.0627451  0.24313726 0.61960787]
 [0.09411765 0.24705882 0.5647059 ]
 [0.07058824 0.24313726 0.5568628 ]
 [0.07058824 0.2509804  0.4627451 ]
 [0.07058824 0.24705882 0.32941177]
 [0.08627451 0.23529412 0.33333334]
 [0.10196079 0.22352941 0.30588236]
 [0.04705882 0.20392157 0.28235295]
 [0.0627451  0.22745098 0.28627452]
 [0.07843138 0.22745098 0.19215687]
 [0.07843138 0.20392157 0.16862746]
 [0.09411765 0.20392157 0.23529412]
 [0.07450981 0.21960784 0.1882353 ]
 [0.07058824 0.20784314 0.09411765]
 [0.12941177 0.2        0.0627451 ]
 [0.23921569 0.20784314 0.11372549]
 [0.27058825 0.20392157 0.18

In [10]:
model_path = os.path.join('models', 'imageclassifier5.h5')
model = load_model(model_path)
prediction = model.predict(all)


1/1 [==============================] - 1s 584ms/step


In [11]:
prediction

array([[0.7348131 , 0.25867304, 0.4119315 ]], dtype=float32)

In [34]:
prediction = prediction[0]
    abnormal_pred = prediction[0]
    acl_pred = prediction[1]
    meniscus_pred = prediction[2]


    if acl_pred > 0.55:
        print("ACL injury")
        print("type ", type(acl_pred))
    elif abnormal_pred > 0.55:
        print("No ACL injury, but has abnormalities")
        
    else:
        print("No ACL injury")

IndentationError: unexpected indent (1431896827.py, line 2)

In [7]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 160, 160, 3)]    0         []                            
                                                                                                  
 conv3d (Conv3D)             (None, 25, 80, 80, 32)       32960     ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 25, 80, 80, 32)       128       ['conv3d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 25, 80, 80, 32)       0         ['batch_normalization[0][0